# 导包

In [1]:
import sys 
sys.version

'3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) \n[GCC 9.3.0]'

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !unzip sohu2021_open_data_clean.zip
# !unzip chinese_L-12_H-768_A-12.zip

In [4]:
# !pip install transformers

In [5]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import re
from collections import Counter
import random
import json

from tqdm import tqdm
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.get_logger().setLevel(tf.compat.v1.logging.ERROR)
from keras.metrics import top_k_categorical_accuracy, binary_accuracy
from keras.layers import *
from keras.callbacks import *
from keras.models import Model, load_model
import keras.backend as K
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.losses import SparseCategoricalCrossentropy, binary_crossentropy
from transformers import (
    BertTokenizer,
    TFBertForPreTraining,
    TFBertModel,
)
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
import torch

In [6]:
tf.__version__

'2.4.1'

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 68250996340265039,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10770692224
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6782669471833480376
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1"]

In [8]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

In [9]:
data_path = "sohu2021_open_data_clean/"
text_max_length = 512
bert_path = r"chinese_L-12_H-768_A-12"

# 构建标签表

In [10]:
label_to_id = {'0':0, '1':1}

In [11]:
labels = [0, 1]

# 构建原数据文本迭代器

In [12]:
def _transform_text(text):
   text = text.strip().replace('\n', '。').replace('\t', '').replace('\u3000', '')
   return re.sub(r'。+', '。', text)

In [13]:
def get_data_iterator(data_path, file_name):
    # TODO: 随机取
    for category in os.listdir(data_path):
        category_path = os.path.join(data_path, category)
        if not os.path.isdir(category_path):
            continue
            
        file_path = os.path.join(category_path, file_name)
        if not os.path.isfile(file_path):
            continue
        
#         print(file_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line)
                
                data['source'] = _transform_text(data['source'])
                if len(data['source']) == 0:
                    print('source:', line, data)
                    break
#                     continue
                    
                data['target'] = _transform_text(data['target'])
                if len(data['target']) == 0:
                    print('target:', line, data)
                    break
#                     continue
                
                label_name_list = list(key for key in data.keys() if key[:5]=='label')
                if len(label_name_list) != 1:
                    print('label_name_list:', line, data)
                    break
#                     continue
                label_name = label_name_list[0]
                if data[label_name] not in label_to_id.keys():
                    print('label_name:', line, data, label_name)
                    break
#                     continue
                    
                yield data['source'], data['target'], label_to_id[data[label_name]]

In [14]:
it = get_data_iterator(data_path, "train.txt")

In [15]:
next(it)

('最近 你有没有约上好友一起去看电影？ 观影的感觉是不是很爽…… 不过 消防蜀黍在这里还得啰嗦几句 电影院属于相对密闭的人员密集场所， 一旦发生火灾 极易造成群死群伤 后果不堪设想 01 电影院火灾案例一 2020年8月21日，广东一电影院突发火灾，现场烟雾弥漫，影院工作人员紧急疏散观众。 02 电影院火灾案例二 2020年9月4日，浙江台州一电影院荧幕突然起火，观众紧急撤离。 电影院一般装修豪华 电气线路敷设复杂 可燃易燃物质较多 如果消防安全管理稍有不慎 就有可能引发火灾事故  电影院的火灾危险性：  由于电影院空间高大，空气对流好，舞台可燃物多，一旦发生火灾，火势极易蔓延扩大。火势会通过舞台的幕布、放映室等各个部位的大量可燃物，迅速向吊顶蔓延而形成立体燃烧。 1   电影院由于可燃物多，特别如化纤、海绵等，火灾发生时能产生大量的有毒气体，造成人员中毒伤亡。 2   电影院人员高度密集，环境封闭，能见度低，有序疏散非常困难。 3  那么，在电影院遇到火灾，又怎么逃生呢？ 作为观影者 我们需要了解一些火灾发生时的逃生方法  观影前，应注意影院所在楼层位置，附近有几个安全疏散通道。如果是深夜观影，应问清工作人员，哪个通道直通楼外。 进入影厅后，注意查看是否有其他安全出口，在发生火灾时，可以分散逃生，查看是否有卫生间。 可以在有浓烟的情况下打湿衣物捂住口鼻，尽量减少有毒气体的吸入。 由于影厅内使用的材质多是可燃物质，发生火灾后，火势蔓延迅速，浓烟生成较快，所以逃生时切忌高声呼喊、上蹿下跳，应尽量保持低姿前行，避免大量吸入有毒气体，导致晕厥甚至窒息。 发生火灾时应沉着冷静对待，在工作人员的组织下，往安全出口或进出口方向快速、有序逃生，切忌堵塞通道，造成不必要的人员伤亡。 如果第一时间未能撤离，应选择火势蔓延相反方向进行躲避，尽量打湿衣物，裹住面部，靠近承重墙蹲下或趴下，选择时机逃生或等待救援。 以上内容你学会了吗？',
 '最近 你有没有约上好友一起去看电影？ 观影的感觉是不是很爽…… 不过 消防蜀黍在这里还得啰嗦几句 电影院属于相对密闭的人员密集场所， 一旦发生火灾 极易造成群死群伤 后果不堪设想 01 电影院火灾案例一 2020年8月21日，广东一电影院突发火灾，现场烟雾弥漫，影院工作人员紧急疏散观众。 02 电影院火灾案例二 2020年9月4日，浙江台州一电影院

# 获取数据集样本个数

In [16]:
def get_sample_num(data_path, file_name):
    count = 0
    it = get_data_iterator(data_path, file_name)
    for data in tqdm(it):
        count += 1
    return count

In [17]:
train_sample_count = get_sample_num(data_path, "train.txt")

59638it [00:02, 21011.81it/s]


In [18]:
dev_sample_count = get_sample_num(data_path, "valid.txt")

9940it [00:00, 19764.08it/s]


In [19]:
train_sample_count, dev_sample_count

(59638, 9940)

# 构建数据迭代器

In [20]:
tokenizer = BertTokenizer.from_pretrained(bert_path)

In [41]:
def _get_indices(text, text_pair=None):
    return tokenizer.encode(text=text,
                            text_pair=text_pair,
                            max_length=text_max_length, 
                            add_special_tokens=True, 
                            padding='max_length', 
#                             truncation_strategy='only_first', 
                            truncation=True,
#                                          return_tensors='tf'
                            )

In [42]:
def get_keras_bert_iterator(data_path, file_name, tokenizer):
    while True:
        data_it = get_data_iterator(data_path, file_name)
        for source, target, label in data_it:
            indices = _get_indices(text=source, 
                                   text_pair=target)
            yield indices, label

In [43]:
it = get_keras_bert_iterator(data_path, "train.txt", tokenizer)

In [44]:
# next(it)

# 构建批次数据迭代器

In [45]:
def batch_iter(data_path, file_name, tokenizer, batch_size=64, shuffle=True):
    """生成批次数据"""
    keras_bert_iter = get_keras_bert_iterator(data_path, file_name, tokenizer)
    while True:
        data_list = []
        for _ in range(batch_size):
            data = next(keras_bert_iter)
            data_list.append(data)
        if shuffle:
            random.shuffle(data_list)
        
        indices_list = []
        label_list = []
        for data in data_list:
            indices, label = data
            indices_list.append(indices)
            label_list.append(label)

        yield np.array(indices_list), np.array(label_list)

In [46]:
it = batch_iter(data_path, "train.txt", tokenizer, batch_size=1)

In [47]:
# next(it)

In [48]:
it = batch_iter(data_path, "train.txt", tokenizer, batch_size=2)

In [49]:
next(it)

(array([[ 101, 3297, 6818, ..., 8024, 4125,  102],
        [ 101, 2218, 5050, ..., 1059, 1377,  102]]),
 array([1, 0]))

In [53]:
# bert_model = TFBertModel.from_pretrained(bert_path, from_pt=True,
#                                          trainable=True,
# #                                          return_dict=True
#                                         )

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [54]:
# it = batch_iter(data_path, "train.txt", tokenizer, batch_size=1)
# data = next(it)
# bert_model(data[0])

TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor: shape=(1, 512, 768), dtype=float32, numpy=
array([[[ 0.81342965,  0.11446221, -1.3161695 , ..., -0.3807172 ,
         -0.11636969,  0.09716403],
        [-0.32854915,  0.4526521 ,  0.44791842, ..., -1.0548452 ,
         -0.4015216 ,  0.07144768],
        [-0.40069616,  0.00229135, -1.2190996 , ...,  0.8750916 ,
          1.0793246 , -0.10630547],
        ...,
        [ 0.12962021, -0.07015229, -1.0330786 , ...,  0.92563105,
         -0.33958027,  0.16176102],
        [ 0.3947732 , -0.303226  , -1.3745954 , ..., -0.30742544,
          0.4627821 , -0.04974386],
        [ 0.0482906 , -0.23322865, -0.48208088, ...,  0.22937274,
          0.22010675, -0.18900992]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[ 0.9980097 , -0.68254197,  0.99999696, -0.90211046, -0.8536456 ,
        -0.80917877,  0.9048096 ,  0.9579247 , -0.5031664 , -0.9998114 ,
         0.9999901 ,  0.99999976,  0.1195591

# 定义base模型

In [30]:
# !transformers-cli convert --model_type bert \
#   --tf_checkpoint chinese_L-12_H-768_A-12/bert_model.ckpt \
#   --config chinese_L-12_H-768_A-12/bert_config.json \
#   --pytorch_dump_output chinese_L-12_H-768_A-12/pytorch_model.bin

In [31]:
# bert_model = TFBertForPreTraining.from_pretrained("./chinese_L-12_H-768_A-12/", from_pt=True)

In [32]:
# # it = get_keras_bert_iterator(r"data/keras_bert_train.txt", cat_to_id, tokenizer)
# it = batch_iter(r"data/keras_bert_train.txt", cat_to_id, tokenizer, batch_size=1)
# out = bert_model(next(it)[0])
# out[0]

In [33]:
def get_model(label_list):
    K.clear_session()
    
    bert_model = TFBertForPreTraining.from_pretrained(bert_path, from_pt=True)
 
    input_indices = Input(shape=(None,), dtype='int32')
 
    bert_output = bert_model(input_indices)
    projection_logits = bert_output[0]
    bert_cls = Lambda(lambda x: x[:, 0])(projection_logits) # 取出[CLS]对应的向量用来做分类
    
    dropout = Dropout(0.5)(bert_cls)
    output = Dense(len(label_list), activation='softmax')(dropout)
 
    model = Model(input_indices, output)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(1e-5),    #用足够小的学习率
                  metrics=['accuracy'])
    print(model.summary())
    return model

In [34]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)   #早停法，防止过拟合
plateau = ReduceLROnPlateau(monitor="val_accuracy", verbose=1, mode='max', factor=0.5, patience=2) #当评价指标不在提升时，减少学习率
checkpoint = ModelCheckpoint('trained_model/keras_bert_sohu.hdf5', monitor='val_loss',verbose=2, save_best_only=True, mode='max', save_weights_only=True) #保存最好的模型

## 模型训练

In [35]:
def get_step(sample_count, batch_size):
    step = sample_count // batch_size
    if sample_count % batch_size != 0:
        step += 1
    return step

In [36]:
# batch_size = 2
# train_step = get_step(train_sample_count, batch_size)
# dev_step = get_step(dev_sample_count, batch_size)

# train_dataset_iterator = batch_iter(data_path, "train.txt", tokenizer, batch_size)
# dev_dataset_iterator = batch_iter(data_path, "valid.txt", tokenizer, batch_size)

# model = get_model(labels)

# #模型训练
# model.fit(
#     train_dataset_iterator,
#     steps_per_epoch=10,
# #     steps_per_epoch=train_step,
#     epochs=5,
#     validation_data=dev_dataset_iterator,
#     validation_steps=2,
# #     validation_steps=dev_step,
#     callbacks=[early_stopping, plateau, checkpoint],
#     verbose=1
# )

# model.save_weights("trained_model/keras_bert_sohu_final.weights")
# model.save("trained_model/keras_bert_sohu_final.model")

# 多任务分支模型

## 构建数据迭代器

In [37]:
label_type_to_id = {'labelA':0, 'labelB':1}

In [38]:
def get_text_iterator(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            yield line

In [39]:
def get_data_iterator(data_path, file_names):
    # TODO: 随机取
    file_iters = []
    for file_name in file_names:
      for category in os.listdir(data_path):
          category_path = os.path.join(data_path, category)
          if not os.path.isdir(category_path):
              continue
              
          file_path = os.path.join(category_path, file_name)
          if not os.path.isfile(file_path):
              continue
              
          
          file_iter = get_text_iterator(file_path)
          cat_source = 0
          if category[0] == '长':
            cat_source = 1
          cat_target = 0
          if category[1] == '长':
            cat_target = 1
          file_iters.append((file_iter, cat_source, cat_target))
        
    while len(file_iters) > 0:
        i = random.randrange(len(file_iters))
        line = next(file_iters[i][0], None)
        cat_source = file_iters[i][1]
        cat_target = file_iters[i][2]
        if line is None:
            del file_iters[i]
            continue
            
        data = json.loads(line)

        data['source'] = _transform_text(data['source'])
        if len(data['source']) == 0:
            print('source:', line, data)
            break
#                     continue

        data['target'] = _transform_text(data['target'])
        if len(data['target']) == 0:
            print('target:', line, data)
            break
#                     continue

        label_name_list = list(key for key in data.keys() if key[:5]=='label')
        if len(label_name_list) != 1:
            print('label_name_list:', line, data)
            break
#                     continue
        label_name = label_name_list[0]
        if data[label_name] not in label_to_id.keys():
            print('label_name:', line, data, label_name)
            break
#                     continue
        
        label_dict = {key: -1 for key in label_type_to_id.keys()}
        label_dict[label_name] = label_to_id[data[label_name]]
        if label_dict['labelA'] == 0:
            label_dict['labelB'] = 0
        if label_dict['labelB'] == 1:
            label_dict['labelA'] = 1

        label_dict['labelC'] = 0
        if label_dict['labelA'] == 1 or label_dict['labelB'] == 1:
          label_dict['labelC'] = 1

        yield data['source'], data['target'], cat_source, cat_target, label_dict['labelA'], label_dict['labelB'], label_dict['labelC']

In [40]:
it = get_data_iterator(data_path, ["train.txt", "valid.txt", "round2.txt"])

In [41]:
next(it)

('48岁吴绮莉现身避谈小龙女，主动提生父，对方墓地买好不愿回港 48岁吴绮莉现身避谈小龙女，主动提生父，对方墓地买好不愿回港 重新回到香港生活的吴绮莉，虽然时不时就会因为小龙女吴卓林的事情上头条，但是这几年慢慢回归事业的她，状态倒是越来越好了。曾经她也说过女儿已经成年了，自己也应该有自己的生活了。  48岁的吴绮莉现身某电视台，为自己即将上映的新片做宣传，当天吴绮莉的造型也是非常的保暖，一身长款的黑衣外套，看起来气色不错。虽然刚带着小龙女回港的那几年，吴绮莉也是非常沧桑，但是经过这几年的保养，状态已经今时不同往日了，毕竟她也曾是港姐冠军，底子还是很好的。  吴绮莉这次宣传的新剧，正是她去年的时候参与拍摄的《太平纹身店》。这部剧杀青也是大半年了，主要还是因为疫情的影响，所以才会一直拖到现在，才开始宣传和上映，这也是吴绮莉复出后的首部影视作品，因此她也是非常看重！  事实上，港姐出身的吴绮莉曾经也像大多数港姐一样，签约了TVB成为演员，后来生下小龙女之后，她除了《雷霆扫军》就没有其他作品了，在离开TVB之后更是选择在电台工作。  去年5月份的时候，吴绮莉选择再次回归演员的身份，但合作的却不是TVB，而且老东家的对手台某TV，拍摄的正是这部《太平纹身店》，虽然吴绮莉已经有七年没有拍过戏了，但她也曾经是一名专业的演员，可以期待一下。  不过相比吴绮莉的新作品，还是她和小龙女的事情更受媒体的关注，我们都知道在小龙女的成长过程中，和吴绮莉的母女关系也是发生很多的变化，特别有了交往和结婚对象之后，母女两的关系更是越来越差。  这次媒体也问起了吴绮莉和小龙女目前的情况，但是吴绮莉表示不想谈和女儿的私事，甚至为了避免谈小龙女的事情，主动提起了自己生父的事情。  前段时间，吴绮莉突然在自己的社交账号上自曝，称已经失联37年的父亲突然给她打了电话，让她又惊又喜。据吴绮莉的描述，从她记事起，这辈子只见过父亲两次，一次是妈妈离婚带走自己的时候，还有一次则是11岁的时候，在街头匆匆的偶遇了一回。  看得出这次久违的联系，父女两的对话也是让人动容，吴绮莉更是没有因为多年不联系而觉得陌生或者排斥，反而有点珍惜这份来之不易的“父爱”。吴绮莉在活动中说道，父亲定居美国多年，但如今身边的亲朋好友们都已经搬走了，只剩下他一个人独居，所以吴绮莉有打算将八十来岁的老父亲接回香港照顾。  只是吴绮莉的父亲似乎

In [42]:
sample_count = get_sample_num(data_path, ["train.txt", "valid.txt", "round2.txt"])
sample_count

119168it [00:06, 19715.02it/s]


119168

In [43]:
def get_sample_y(data_path, file_names):
    labelA_list = []
    labelB_list = []
    labelC_list = []
    it = get_data_iterator(data_path, file_names)
    for source, target, cat_source, cat_target, labelA, labelB, labelC in tqdm(it):
        if labelA != -1:
          labelA_list.append(labelA)
        if labelB != -1:
          labelB_list.append(labelB)
        labelC_list.append(labelC)
    return labelA_list, labelB_list, labelC_list

In [44]:
# labelA_list, labelB_list, labelC_list = get_sample_y(data_path, ["train.txt", "valid.txt"])
# labelA_class_weights = class_weight.compute_class_weight('balanced', np.unique(labelA_list), labelA_list)
# labelB_class_weights = class_weight.compute_class_weight('balanced', np.unique(labelB_list), labelB_list)
# labelC_class_weights = class_weight.compute_class_weight('balanced', np.unique(labelC_list), labelC_list)
# labelA_class_weights, labelB_class_weights, labelC_class_weights

In [45]:
def _get_indices(text, text_pair=None):
    return tokenizer.encode_plus(text=text,
                            text_pair=text_pair,
                            max_length=text_max_length, 
                            add_special_tokens=True, 
                            padding='max_length', 
#                             truncation_strategy='longest_first', 
                            truncation=True,
#                                          return_tensors='tf',
                            return_token_type_ids=True
                            )

In [46]:
def get_keras_bert_iterator(data_path, file_names, tokenizer):
    while True:
        data_it = get_data_iterator(data_path, file_names)
        for source, target, cat_source, cat_target, labelA, labelB, labelC in data_it:
            data_source = _get_indices(text=source)
            data_target = _get_indices(text=target)
#             print(indices, type(indices), len(indices))
            yield data_source['input_ids'], data_source['token_type_ids'], data_source['attention_mask'], data_target['input_ids'], data_target['token_type_ids'], data_target['attention_mask'], cat_source, cat_target, labelA, labelB, labelC

In [47]:
it = get_keras_bert_iterator(data_path, ["train.txt", "valid.txt", "round2.txt"], tokenizer)

In [48]:
# next(it)

In [49]:
def batch_iter(data_path, file_names, tokenizer, batch_size=64, shuffle=True):
    """生成批次数据"""
    keras_bert_iter = get_keras_bert_iterator(data_path, file_names, tokenizer)
    while True:
        data_list = []
        for _ in range(batch_size):
            data = next(keras_bert_iter)
            data_list.append(data)
        if shuffle:
            random.shuffle(data_list)
        
        input_ids_texta_list = []
        token_type_ids_texta_list = []
        attention_mask_texta_list = []
        input_ids_textb_list = []
        token_type_ids_textb_list = []
        attention_mask_textb_list = []
        cat_texta_list = []
        cat_textb_list = []
        labelA_list = []
        labelB_list = []
        labelC_list = []
        for data in data_list:
            input_ids_texta, token_type_ids_texta, attention_mask_texta, input_ids_textb, token_type_ids_textb, attention_mask_textb, cat_texta, cat_textb, labelA, labelB, labelC = data
#             print(indices, type(indices))
            input_ids_texta_list.append(input_ids_texta)
            token_type_ids_texta_list.append(token_type_ids_texta)
            attention_mask_texta_list.append(attention_mask_texta)
            input_ids_textb_list.append(input_ids_textb)
            token_type_ids_textb_list.append(token_type_ids_textb)
            attention_mask_textb_list.append(attention_mask_textb)
            cat_texta_list.append(cat_texta)
            cat_textb_list.append(cat_textb)
            labelA_list.append(labelA)
            labelB_list.append(labelB)
            labelC_list.append(labelC)

        yield [np.array(input_ids_texta_list), np.array(token_type_ids_texta_list), np.array(attention_mask_texta_list), 
               np.array(input_ids_textb_list), np.array(token_type_ids_textb_list), np.array(attention_mask_textb_list), 
               np.array(cat_texta_list), np.array(cat_textb_list)], \
            [np.array(labelA_list, dtype=np.int32), np.array(labelB_list, dtype=np.int32), np.array(labelC_list, dtype=np.int32)]

In [50]:
it = batch_iter(data_path, ["train.txt", "valid.txt", "round2.txt"], tokenizer, batch_size=2)

In [51]:
next(it)

([array([[ 101, 7716, 1305, ...,    0,    0,    0],
         [ 101,  523, 3217, ...,    0,    0,    0]]),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]),
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]]),
  array([[ 101,  791, 1921, ..., 3209, 8024,  102],
         [ 101,  517, 1173, ...,    0,    0,    0]]),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]),
  array([[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 0, 0, 0]]),
  array([0, 0]),
  array([1, 0])],
 [array([0, 1], dtype=int32),
  array([ 0, -1], dtype=int32),
  array([0, 1], dtype=int32)])

In [52]:
def get_test_data_iterator(data_path, file_name):
  # print(data_path)
  for category in os.listdir(data_path):
    category_path = os.path.join(data_path, category)
    # print(category_path)
    if not os.path.isdir(category_path):
      # print(f"{category_path} not dir")
      continue
        
    file_path = os.path.join(category_path, file_name)
    # print(file_path)
    if not os.path.isfile(file_path):
      # print(f"{file_path} not file")
      continue
    
    with open(file_path, 'r', encoding='utf-8') as f:
      for line in f:
        # print(line)
        data = json.loads(line)
        
        data['source'] = _transform_text(data['source'])
        if len(data['source']) == 0:
          print('source:', line, data)
          break
            
        data['target'] = _transform_text(data['target'])
        if len(data['target']) == 0:
          print('target:', line, data)
          break

        cat_source = 0
        if category[0] == '长':
          cat_source = 1
        cat_target = 0
        if category[1] == '长':
          cat_target = 1
            
        yield data['source'], data['target'], cat_source, cat_target, data['id']

In [53]:
def get_test_keras_bert_iterator(data_path, file_name):
  it = get_test_data_iterator(data_path, file_name)
  for source, target, cat_source, cat_target, id in it:
    data_source = _get_indices(text=source)
    data_target = _get_indices(text=target)
    yield data_source['input_ids'], data_source['token_type_ids'], data_source['attention_mask'], data_target['input_ids'], data_target['token_type_ids'], data_target['attention_mask'], cat_source, cat_target, id

In [54]:
def get_test_iterator(data_path, file_name):
  it = get_test_keras_bert_iterator(data_path, file_name)
  for input_ids_texta, token_type_ids_texta, attention_mask_texta, input_ids_textb, token_type_ids_textb, attention_mask_textb, cat_source, cat_target, id in it:
    yield [np.array([input_ids_texta]), np.array([token_type_ids_texta]), np.array([attention_mask_texta]), np.array([input_ids_textb]), np.array([token_type_ids_textb]), np.array([attention_mask_textb]), np.array([cat_source]), np.array([cat_target])], id

In [55]:
it = get_test_iterator(data_path, "test_with_id.txt")
# next(it)

In [56]:
def save_test_result(model, result_file, result_file_1):
  it = get_test_iterator(data_path, "test_with_id.txt")
#   print("      ", end="")
#   count = 0
  with open(result_file, 'w', encoding='utf-8') as f, open(result_file_1, 'w', encoding='utf-8') as f_1:
    f.write(f"id,label\n")
    f_1.write(f"id,label\n")
    for data, id in tqdm(it):
      predict = model.predict(data)
      if id[-1] == 'a':
        predict_cls = 1 if predict[0][0][0] > 0.5 else 0
      elif id[-1] == 'b':
        predict_cls = 1 if predict[1][0][0] > 0.5 else 0
      else:
        print(id)
        continue
      f.write(f"{id},{predict_cls}\n")
      
      predict_cls_1 = 0
      if predict[2][0][0] > 0.5:
            predict_cls_1 = 1
      f_1.write(f"{id},{predict_cls_1}\n")
#       count += 1
#       print(f"\b\b\b\b\b\b{count}", end="")

## 定义模型

In [57]:
def variant_focal_loss(gamma=2., alpha=0.5, rescale = False):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        # print(y_true)
        """
        Focal loss for bianry-classification
        FL(p_t)=-rescaled_factor*alpha_t*(1-p_t)^{gamma}log(p_t)
        
        Notice: 
        y_pred is probability after sigmoid

        Arguments:
            y_true {tensor} -- groud truth label, shape of [batch_size, 1]
            y_pred {tensor} -- predicted label, shape of [batch_size, 1]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})  
            alpha {float} -- (default: {0.5})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9  
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        model_out = tf.clip_by_value(y_pred, epsilon, 1.-epsilon)  # to advoid numeric underflow
        
        # compute cross entropy ce = ce_0 + ce_1 = - (1-y)*log(1-y_hat) - y*log(y_hat)
        ce_0 = tf.multiply(tf.subtract(1., y_true), -tf.math.log(tf.subtract(1., model_out)))
        ce_1 = tf.multiply(y_true, -tf.math.log(model_out))

        # compute focal loss fl = fl_0 + fl_1
        # obviously fl < ce because of the down-weighting, we can fix it by rescaling
        # fl_0 = -(1-y_true)*(1-alpha)*((y_hat)^gamma)*log(1-y_hat) = (1-alpha)*((y_hat)^gamma)*ce_0
        fl_0 = tf.multiply(tf.pow(model_out, gamma), ce_0)
        fl_0 = tf.multiply(1.-alpha, fl_0)
        # fl_1= -y_true*alpha*((1-y_hat)^gamma)*log(y_hat) = alpha*((1-y_hat)^gamma*ce_1
        fl_1 = tf.multiply(tf.pow(tf.subtract(1., model_out), gamma), ce_1)
        fl_1 = tf.multiply(alpha, fl_1)
        fl = tf.add(fl_0, fl_1)
        f1_avg = tf.reduce_mean(fl)
        
        if rescale:
            # rescale f1 to keep the quantity as ce
            ce = tf.add(ce_0, ce_1)
            ce_avg = tf.reduce_mean(ce)
            rescaled_factor = tf.divide(ce_avg, f1_avg + epsilon)
            f1_avg = tf.multiply(rescaled_factor, f1_avg)
        
        return f1_avg
    
    return focal_loss_fixed

In [58]:
def f1_loss(y_true, y_pred):
    # y_true:真实标签0或者1；y_pred:为正类的概率
    loss = 2 * tf.reduce_sum(y_true * y_pred) / tf.reduce_sum(y_true + y_pred) + K.epsilon()
    return -loss

In [59]:
def transform_y(y_true, y_pred):
    mask_value = tf.constant(-1)
    mask_y_true = tf.not_equal(tf.cast(y_true, dtype=tf.int32), tf.cast(mask_value, dtype=tf.int32))
#     print(f"mask_y_true:{mask_y_true}")
#     y_true_ = tf.cond(tf.equal(y_true, mask_value), lambda: 0, lambda: y_true)
    y_true_ = tf.cast(y_true, dtype=tf.int32) * tf.cast(mask_y_true, dtype=tf.int32)
    y_pred_ = tf.cast(y_pred, dtype=tf.float32) * tf.cast(mask_y_true, dtype=tf.float32)
#     print(f"y_true_:{y_true_}, y_pred_:{y_pred_}")
    
    return tf.cast(y_true_, dtype=tf.float32), tf.cast(y_pred_, dtype=tf.float32)

In [60]:
def my_binary_crossentropy(y_true, y_pred):
    # print(f"y_true:{y_true}, y_pred:{y_pred}")
    
    y_true, y_pred = transform_y(y_true, y_pred)
    # print(f"y_true_:{y_true}, y_pred_:{y_pred}")

    # loss = binary_crossentropy(y_true, y_pred)
    loss = variant_focal_loss()(y_true, y_pred)
#     loss = f1_loss(y_true, y_pred)

    # y_true0 = tf.constant([y_true.numpy()[0]])
    # y_true1 = tf.constant([y_true.numpy()[1]])
    # y_pred0 = tf.constant([y_pred.numpy()[0]])
    # y_pred1 = tf.constant([y_pred.numpy()[1]])
    # loss0 = variant_focal_loss()(y_true0, y_pred0)
    # loss1 = variant_focal_loss()(y_true1, y_pred1)
    # print(f"y_true_0:{y_true0}, y_pred_0:{y_pred0}")
    # print(f"y_true_1:{y_true1}, y_pred_1:{y_pred1}")
    # print(f"loss0:{loss0}")
    # print(f"loss1:{loss1}")

    # print(f"loss:{loss}")
    return loss

In [61]:
def tarnsform_metrics(y_true, y_pred):
    y_true_, y_pred_ = y_true.numpy(), y_pred.numpy()
    for i in range(y_true_.shape[0]):
        for j in range(y_true_.shape[1]):
            if y_true_[i][j] == -1:
                y_true_[i][j] = 0
                y_pred_[i][j] = random.choice([0, 1])
            if y_pred_[i][j] > 0.5:
                y_pred_[i][j] = 1
            else:
                y_pred_[i][j] = 0
    return y_true_, y_pred_

In [62]:
def my_binary_accuracy(y_true, y_pred):
#     print("my_binary_accuracy")
#     print(f"y_true:{y_true}, y_pred:{y_pred}")
    
    y_true_, y_pred_ = tarnsform_metrics(y_true, y_pred)
#     print(f"y_true_:{y_true_}, y_pred_:{y_pred_}")

    accuracy = binary_accuracy(y_true_, y_pred_)
    return accuracy

In [63]:
def my_f1_score(y_true, y_pred):
#     print("my_f1_score")
#     print(f"y_true:{y_true}, y_pred:{y_pred}")
    
    y_true_, y_pred_ = tarnsform_metrics(y_true, y_pred)
#     print(f"y_true_:{y_true_}, y_pred_:{y_pred_}")

    return f1_score(y_true_, y_pred_, average='macro')

In [64]:
# bert_model = TFBertForPreTraining.from_pretrained(bert_path, from_pt=True)
# data = next(it)[0]
# bert_output_texta = bert_model({'input_ids':data[0], 'token_type_ids':data[1], 'attention_mask':data[2]}, return_dict=False, training=True)
# bert_output_texta[0]

In [65]:
def get_model():
    K.clear_session()
    
    bert_model = TFBertModel.from_pretrained(bert_path, from_pt=True, trainable=True)
    for l in bert_model.layers:
        l.trainable = True
 
    input_ids_texta = Input(shape=(None,), dtype='int32')
    input_token_type_ids_texta = Input(shape=(None,), dtype='int32')
    input_attention_mask_texta = Input(shape=(None,), dtype='int32')
    input_ids_textb = Input(shape=(None,), dtype='int32')
    input_token_type_ids_textb = Input(shape=(None,), dtype='int32')
    input_attention_mask_textb = Input(shape=(None,), dtype='int32')
    input_token_type_ids_textb = Input(shape=(None,), dtype='int32')
    input_cat_texta = Input(shape=(1), dtype='float32')
    input_cat_textb = Input(shape=(1), dtype='float32')
 
    bert_output_texta = bert_model({'input_ids':input_ids_texta, 'token_type_ids':input_token_type_ids_texta, 'attention_mask':input_attention_mask_texta}, return_dict=False, training=True)
    projection_logits_texta = bert_output_texta[0]
    bert_cls_texta = Lambda(lambda x: x[:, 0])(projection_logits_texta) # 取出[CLS]对应的向量用来做分类

    bert_output_textb = bert_model({'input_ids':input_ids_textb, 'token_type_ids':input_token_type_ids_textb, 'attention_mask':input_attention_mask_textb}, return_dict=False, training=True)
    projection_logits_textb = bert_output_textb[0]
    bert_cls_textb = Lambda(lambda x: x[:, 0])(projection_logits_textb) # 取出[CLS]对应的向量用来做分类

    bert_cls = concatenate([bert_cls_texta, bert_cls_textb, input_cat_texta, input_cat_textb], axis=-1)
    
    dropout_A = Dropout(0.5)(bert_cls)
    output_A = Dense(1, activation='sigmoid')(dropout_A)
    
    dropout_B = Dropout(0.5)(bert_cls)
    output_B = Dense(1, activation='sigmoid')(dropout_B)

    dropout_C = Dropout(0.5)(bert_cls)
    output_C = Dense(1, activation='sigmoid')(dropout_C)
 
    model = Model([input_ids_texta, input_token_type_ids_texta, input_attention_mask_texta, input_ids_textb, input_token_type_ids_textb, input_attention_mask_textb, input_cat_texta, input_cat_textb], [output_A, output_B, output_C])
    model.compile(
                  loss=[my_binary_crossentropy, my_binary_crossentropy, 'binary_crossentropy'],
#                   loss='binary_crossentropy',
#                   loss=binary_crossentropy,
                  optimizer=Adam(1e-5),    #用足够小的学习率
                  metrics=[my_binary_accuracy, my_f1_score]
#                   metrics='accuracy'
                 )
    print(model.summary())
    return model

In [66]:
early_stopping = EarlyStopping(monitor='loss', patience=3)   #早停法，防止过拟合
plateau = ReduceLROnPlateau(monitor="loss", verbose=1, mode='max', factor=0.5, patience=2) #当评价指标不在提升时，减少学习率
checkpoint = ModelCheckpoint('trained_model/multi_keras_bert_sohu.weights', monitor='loss', verbose=2, save_best_only=True, save_weights_only=True) #保存最好的模型
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", update_freq=5000)

## 模型训练

In [67]:
# batch_size = 2

# train_dataset_iterator = batch_iter(data_path, ["train.txt", "valid.txt", "round2.txt"], tokenizer, batch_size)
# train_step = get_step(sample_count, batch_size)

# model = get_model()

# model.fit(
#   train_dataset_iterator,
#   # steps_per_epoch=10,
#   steps_per_epoch=train_step,
#   epochs=5,
# #       validation_data=dev_dataset_iterator,
#   # validation_steps=2,
# #       validation_steps=dev_step,
# #   callbacks=[early_stopping, plateau, checkpoint, tensorboard_callback],
#   verbose=1
# )

# model.save_weights(f"trained_model/multi_keras_bert_sohu_final.weights")

# save_test_result(model, f"trained_model/multi_keras_bert_sohu_test_result_final.csv", f"trained_model/multi_keras_bert_sohu_test_result_1.csv")

In [68]:
model = get_model()
model.load_weights('trained_model/multi_keras_bert_sohu.weights')

All PyTorch model weights were used when initializing TFBertForPreTraining.

All the weights of TFBertForPreTraining were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [69]:
save_test_result(model, 
                 f"trained_model/multi_keras_bert_sohu_test_result_.csv",
                 f"trained_model/multi_keras_bert_sohu_test_result_1.csv")

0it [00:00, ?it/s]/data1/wangchenyue/zsd/code/sohu2021/conda_env/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "
29822it [5:50:52,  1.42it/s]


In [70]:
# epoch = 1
# per_train_step = 5000
# per_epoch_step = (sample_count // batch_size + 1) // per_train_step + 1

# # start_i = (3+8+1+2) % per_epoch_step
# start_i = (0+1) % per_epoch_step
# for i in range(start_i):
#   for j in range(per_train_step):
#     next(train_dataset_iterator)


# #模型训练
# for epoch_ in range(epoch):
#   for i in range(start_i, per_epoch_step):
#     model.fit(
#         train_dataset_iterator,
#         steps_per_epoch=per_train_step,
#         epochs=1,
#         # class_weight=[labelA_class_weights, labelB_class_weights, labelC_class_weights],
#         verbose=1
#     )

#     model.save_weights(f"/content/drive/MyDrive/sohu_match/multi_keras_bert_sohu_step.weights")
#     # model.save(f"/content/drive/MyDrive/sohu_match/multi_keras_bert_sohu_step.model")

#   save_test_result(model, f"/content/drive/MyDrive/sohu_match/multi_keras_bert_sohu_test_result_epoch.csv", f"trained_model/multi_keras_bert_sohu_test_result_1.csv")

In [71]:
# dev_dataset_iterator = batch_iter(data_path, "valid.txt", tokenizer, batch_size=1)
# data = next(dev_dataset_iterator)
# model.predict(data[0]), data[1]

In [72]:
# data = next(dev_dataset_iterator)
# model.predict(data[0]), data[1]

In [73]:
# model.save_weights("/content/drive/MyDrive/multi_keras_bert_sohu_final.weights")
# model.save("/content/drive/MyDrive/multi_keras_bert_sohu_final.model")

# 模型加载及测试

## load_weights

## load_model